<a href="https://colab.research.google.com/github/hfnvbh/colab-trt-pose/blob/master/trt-pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget -q https://www.dropbox.com/s/p0sfni1dzb359hk/nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb
!dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb
!apt-key add /var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427/7fa2af80.pub

In [0]:
!apt-get install -y --no-install-recommends libnvinfer5=5.1.5-1+cuda10.1 libnvinfer-dev=5.1.5-1+cuda10.1
!apt-get update
!apt-get install tensorrt
!apt-get install -y --no-install-recommends python3-libnvinfer=5.1.5-1+cuda10.1 python3-libnvinfer-dev=5.1.5-1+cuda10.1

In [0]:
!dpkg -l | grep TensorRT

In [0]:
!pip3 install tqdm cython pycocotools
!apt-get install python3-matplotlib
!git clone https://github.com/NVIDIA-AI-IOT/trt_pose
!cd trt_pose && python3 setup.py install

In [0]:
!git clone https://github.com/NVIDIA-AI-IOT/torch2trt
!cd torch2trt && python setup.py install

In [0]:
!git clone https://github.com/hfnvbh/colab-trt-pose

In [0]:
!pip install -q youtube-dl visdom
from IPython.display import YouTubeVideo

In [0]:
!wget -q https://www.dropbox.com/s/r7ex76ofqsnjkjy/resnet18_baseline_att_224x224_A_epoch_249.pth

In [0]:
!cp trt_pose/tasks/human_pose/human_pose.json .

In [0]:
YOUTUBE_ID = 'RXABo9hm8B8'
YouTubeVideo(YOUTUBE_ID)

In [0]:
!rm -df youtube.mp4
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

In [0]:
!ffmpeg -y -loglevel error -i youtube.mp4 -t 5 video.mp4

In [0]:
!python colab-trt-pose/convert_model_to_trt.py

In [0]:
!python colab-trt-pose/process_video_trt_pose.py video.mp4

In [0]:
!ffmpeg -y -loglevel error -i output.avi output.mp4 

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4')